In [ ]:
import sys
from torch.utils.data import random_split, DataLoader 
sys.path.append("../")

In [ ]:
from datasets import datasets
import constants
import pickle

Loading images from CVAT source

In [ ]:
images = datasets.FaceRecognitionDataset(images=[], labels=[])

Splitting data into training and testing sets

In [ ]:
train_size = int(len(images) * 0.7)
test_size = int(len(images) * 0.3)
train_dataset, test_dataset = random_split(images, [train_size, test_size])

In [ ]:
training_set = DataLoader(
    dataset=train_dataset, 
    batch_size=constants.BATCH_SIZE,
    shuffle=True
)

testing_set = DataLoader(
    dataset=test_dataset,
    batch_size=constants.BATCH_SIZE,
    shuffle=True
)

Saving datasets to pickle format

In [ ]:
pickle.dumps(training_set, "../data/augmented_data/training_loader.pkl")
pickle.dumps(testing_set, "../data/augmented_data/testing_loader.pkl")